In [1]:
%pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00


In [2]:
from transformers import MarianMTModel, MarianTokenizer, MarianConfig

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-de-en")
config = MarianConfig.from_pretrained("Helsinki-NLP/opus-mt-de-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [3]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [4]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import StepLR

model.to(device)

print(model)

optimizer = optim.Adam(model.parameters(), lr=1e-5)

# criterion = CrossEntropyLoss()

scheduler = StepLR(optimizer, step_size=5, gamma=0.5)


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(58101, 512, padding_idx=58100)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(58101, 512, padding_idx=58100)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [5]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [6]:
sub = 10000
train_tokenized = torch.load("train_tokenized_{}.pth".format(sub)).to(device)

FileNotFoundError: ignored

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(1)
data_set = TensorDataset(train_tokenized.input_ids, train_tokenized.attention_mask, train_tokenized.labels)
data_loader = DataLoader(data_set, batch_size=32)


In [ ]:
references = torch.load('references_{}.pth'.format(sub))

In [ ]:
model.eval()
output_sentences = []
i = 0

for batch in data_loader:
    i+=1
    print(i, '/', len(data_loader))
    output_ids = model.generate(
        input_ids = batch[0],
        attention_mask = batch[1],
        labels = batch[2])
    output_sentences.extend(tokenizer.batch_decode(output_ids, skip_special_tokens=True))

# 10min, 13G/15G


In [ ]:
bleu_scores = []
for i, hypothesis in enumerate(output_sentences):
    bleu_score = sentence_bleu(references=[references[i]], hypothesis=hypothesis)
    bleu_scores.append(round(bleu_score, 4))

In [ ]:
bleu_scores

In [ ]:
idx = bleu_scores.index(0)
output_sentences[idx], references[idx]

In [ ]:
torch.save(bleu_scores, 'bleu_scores_{}.pth'.format(sub))
torch.save(output_sentences, 'output_senetences_{}.pth'.format(sub))